# Main Script

In [1]:
import os
import argparse
from pathlib import Path
from collections import OrderedDict
from tqdm import tqdm
import torchvision
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from scipy.spatial.distance import cdist
import sklearn
import sklearn.covariance
from PIL import Image
from torchvision.models import VGG16_BN_Weights
from torchvision import models, transforms
from numpy.lib.format import open_memmap
import gc

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
to_tensor = transforms.ToTensor()

def load_clean_image_tensor(rel_path: str):
    p = (CLEAN_BASE / str(rel_path).lstrip("/"))
    img = Image.open(p).convert("RGB")
    return to_tensor(img)  

def load_adv_tensor_from_file(rel_path: str, adv_folder: Path):
    p = adv_folder / str(rel_path).lstrip("/").replace(".jpg", ".pt")
    if not p.exists():
        raise FileNotFoundError(f"Adv file not found: {p}")
    t = torch.load(p) 
    if isinstance(t, tuple) or isinstance(t, list):
        t = t[0]
    if t.dim() == 4 and t.shape[0] == 1:
        t = t.squeeze(0)
    return t

def build_stacks_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    adv_folder = ADV_BASE_ROOT / attack_method

    clean_tensors = []
    adv_tensors = []
    rel_paths = []
    skipped = 0

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Loading images and advs"):
        rel = row["rel_path"]
        try:
            clean_t = load_clean_image_tensor(rel)
        except Exception as e:
            print(f"Skip clean {rel}: {e}")
            skipped += 1
            continue

        try:
            adv_t = load_adv_tensor_from_file(rel, adv_folder)
        except Exception as e:
            print(f"Skip adv {rel}: {e}")
            skipped += 1
            continue

        if clean_t.shape != adv_t.shape:
            print ('Missmatch')

        clean_tensors.append(clean_t)
        adv_tensors.append(adv_t)
        rel_paths.append(rel)

    if len(clean_tensors) == 0:
        raise RuntimeError("No pairs loaded. Check CSV paths and adv folder.")

    images_stack = torch.stack(clean_tensors)   
    advs_stack = torch.stack(adv_tensors)       
    print(f"Built stacks: {len(clean_tensors)} pairs (skipped {skipped}).")
    return images_stack, advs_stack, rel_paths


def load_model_for_net(checkpoint_path, strict=True):
    print("Loading model...")
    
    model = models.vgg16_bn(weights=VGG16_BN_Weights.IMAGENET1K_V1)
    model.avgpool = nn.AdaptiveAvgPool2d((7,7))
    model.classifier[6] = nn.Linear(4096, 7)
    
    ckpt = torch.load(checkpoint_path, map_location='cpu')  

    if isinstance(ckpt, dict):
        for candidate in ("model_state", "model_state_dict", "state_dict", "state"):
            if candidate in ckpt:
                state = ckpt[candidate]
                break
        else:
            if all(isinstance(v, (torch.Tensor, type(None))) or hasattr(v, "shape") for v in ckpt.values()):
                state = ckpt
            else:
                nested = None
                for v in ckpt.values():
                    if isinstance(v, dict):
                        if nested is None or len(v) > len(nested):
                            nested = v
                state = nested if nested is not None else ckpt
    else:
        state = ckpt

    if not isinstance(state, dict):
        raise ValueError(f"Checkpoint {checkpoint_path} does not contain a state-dict (found type: {type(state)})")

    keys = list(state.keys())
    prefix = None
    for p in ("module.", "model."):
        cnt = sum(1 for k in keys if k.startswith(p))
        if cnt >= max(1, len(keys) // 2):
            prefix = p
            break

    if prefix:
        new_state = OrderedDict((k[len(prefix):], v) for k, v in state.items())
    else:
        new_state = state

    try:
        model.load_state_dict(new_state, strict=strict)
        print(f"Loaded checkpoint {checkpoint_path} (strict={strict}).")
    except Exception as e:
        print(f"Strict load failed: {e}. Trying non-strict load ...")
        res = model.load_state_dict(new_state, strict=False)
        print("Loaded with strict=False. Missing keys:", getattr(res, "missing_keys", None))
        print("Unexpected keys:", getattr(res, "unexpected_keys", None))

    model = model.to(device).eval()
    return model

def get_layer_feature_maps(X, layers, model_features):
    X_l = []
    X = X.to(device)
    for i in range(len(model_features)):
        X = model_features[i](X)
        if i in layers:
            X_l.append(X)
    return X_l

def cifar_normalize(images, net_name='inat'):
    if net_name == 'cif10':
        images[:,0,:,:] = (images[:,0,:,:] - 0.4914)/0.2023
        images[:,1,:,:] = (images[:,1,:,:] - 0.4822)/0.1994
        images[:,2,:,:] = (images[:,2,:,:] - 0.4465)/0.2010
    elif net_name == 'cif100':
        images[:,0,:,:] = (images[:,0,:,:] - 0.5071)/0.2675
        images[:,1,:,:] = (images[:,1,:,:] - 0.4867)/0.2565
        images[:,2,:,:] = (images[:,2,:,:] - 0.4408)/0.2761
    else:
        MEAN = [0.4812775254249573, 0.4674863815307617, 0.4093940854072571]
        STD  = [0.19709135591983795, 0.1933959424495697, 0.19051066040992737]
        images[:,0,:,:] = (images[:,0,:,:] - MEAN[0]) / STD[0]
        images[:,1,:,:] = (images[:,1,:,:] - MEAN[1]) / STD[1]
        images[:,2,:,:] = (images[:,2,:,:] - MEAN[2]) / STD[2]
    return images

def calculate_fourier_spectrum(im, typ='MFS'):
    im = im.float()
    im = im.cpu().numpy()
    fft = np.fft.fft2(im)
    if typ == 'MFS':
        fourier_spectrum = np.abs(fft)
    elif typ == 'PFS':
        fourier_spectrum = np.abs(np.angle(fft))
    if net == 'cif100' and (attack_method in ('cw','df')):
        fourier_spectrum *= 1/np.max(fourier_spectrum)
    return fourier_spectrum

def calculate_spectra(images, typ='MFS'):
    fs = []
    if isinstance(images, torch.Tensor):
        images = [images[i] for i in range(images.shape[0])]
    for i in range(len(images)):
        image = images[i]
        fourier_image = calculate_fourier_spectrum(image, typ=typ)
        fs.append(fourier_image.flatten())
    return fs

def mle_batch(data, batch, k):
    data = np.asarray(data, dtype=np.float32)
    batch = np.asarray(batch, dtype=np.float32)
    k = min(k, len(data)-1)
    f = lambda v: - k / np.sum(np.log(v/v[-1]))
    a = cdist(batch, data)
    a = np.apply_along_axis(np.sort, axis=1, arr=a)[:,1:k+1]
    a = np.apply_along_axis(f, axis=1, arr=a)
    return a

def extract_characteristics_from_stacks(images, images_advs, model, detector_name, net_name):
    number_images = images.shape[0]

    model_features = model.features
    act_layers= [2,5,9,12,16,19,22,26,29,32,36,39,42]
    fourier_act_layers = [9,16,22,29,36,42]

    print('Extracting ' + detector_name + ' characteristic...')

    if detector_name == 'InputMFS':
        mfs = calculate_spectra(images)
        mfs_advs = calculate_spectra(images_advs)
        characteristics       = np.asarray(mfs, dtype=np.float32)
        characteristics_adv   = np.asarray(mfs_advs, dtype=np.float32)

    elif detector_name == 'InputPFS':
        pfs = calculate_spectra(images, typ='PFS')
        pfs_advs = calculate_spectra(images_advs, typ='PFS')
        characteristics       = np.asarray(pfs, dtype=np.float32)
        characteristics_adv   = np.asarray(pfs_advs, dtype=np.float32)

    elif detector_name == 'LayerMFS':
        mfs = []
        mfs_advs = []
        if net_name == 'cif100' and (attack_method in ('cw','df')):
            layers = [42]
        else:
            layers = fourier_act_layers
        for i in tqdm(range(number_images)):
            image = images[i].unsqueeze_(0)
            adv = images_advs[i].unsqueeze_(0)
            image = cifar_normalize(image, net_name)
            adv = cifar_normalize(adv, net_name)
            image_feature_maps = get_layer_feature_maps(image.to(device), layers, model_features)
            adv_feature_maps = get_layer_feature_maps(adv.to(device), layers, model_features)
            fourier_maps = calculate_spectra(image_feature_maps)
            fourier_maps_adv = calculate_spectra(adv_feature_maps)
            mfs.append(np.hstack(fourier_maps))
            mfs_advs.append(np.hstack(fourier_maps_adv))
        characteristics       = np.asarray(mfs, dtype=np.float32)
        characteristics_adv   = np.asarray(mfs_advs, dtype=np.float32)

    elif detector_name == 'LayerPFS':
        pfs = []
        pfs_advs = []
        if net_name == 'cif100' and (attack_method in ('cw','df')):
            layers = [42]
        else:
            layers = fourier_act_layers
        for i in tqdm(range(number_images)):
            image = images[i].unsqueeze_(0)
            adv = images_advs[i].unsqueeze_(0)
            image = cifar_normalize(image, net_name)
            adv = cifar_normalize(adv, net_name)
            with torch.no_grad(): 
                image_feature_maps = get_layer_feature_maps(image.to(device), layers, model_features)
                adv_feature_maps = get_layer_feature_maps(adv.to(device), layers, model_features)
            fourier_maps = calculate_spectra(image_feature_maps, typ='PFS')
            fourier_maps_adv = calculate_spectra(adv_feature_maps, typ='PFS')
            pfs.append(np.hstack(fourier_maps))
            pfs_advs.append(np.hstack(fourier_maps_adv))
        characteristics       = np.asarray(pfs, dtype=np.float32)
        characteristics_adv   = np.asarray(pfs_advs, dtype=np.float32)

    else:
        raise ValueError("unknown detector")

    return characteristics, characteristics_adv

In [2]:
def extract(IMG_SIZE, CSV_PATH, CLEAN_BASE, ADV_BASE_ROOT, TRAIN_CSV_FOR_MAH,
            attack_method, detector, net):
    OUT_CHAR_DIR = Path(f"./data/characteristics/{IMG_SIZE}/")
    OUT_CHAR_DIR.mkdir(parents=True, exist_ok=True)
    
    print("Building stacks from CSV...")
    images_stack, advs_stack, rels = build_stacks_from_csv(CSV_PATH)
    
    assert images_stack.shape == advs_stack.shape, "Clean/adversarial stacks must have same shape"
    print("Stack shapes:", images_stack.shape)
    
    model = load_model_for_net(f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth')
    
    characteristics, characteristics_adv = extract_characteristics_from_stacks(images_stack, advs_stack, model, detector, net)
    
    prefix = f"{net}_{attack_method}_{detector}"
    np.save(str(OUT_CHAR_DIR / (prefix + f"_{IMG_SIZE}")), characteristics)
    np.save(str(OUT_CHAR_DIR / (prefix + f"_{IMG_SIZE}_adv")), characteristics_adv)
    print("Saved characteristics:", OUT_CHAR_DIR / prefix)
    print("Saved characteristics adv:", OUT_CHAR_DIR / (prefix + "_adv"))
    print("Done.")

In [6]:
IMG_SIZE = 32
CSV_PATH = Path("../02-AdvGenerate/Evaluate/common_success_details.csv")
CLEAN_BASE = Path(f"../01-CleanModel/Dataset/{IMG_SIZE}x{IMG_SIZE}")
ADV_BASE_ROOT = Path(f"../02-AdvGenerate/generated_images-test/{IMG_SIZE}x{IMG_SIZE}")
TRAIN_CSV_FOR_MAH = Path("../01-CleanModel/Dataset/clean_model_train.csv") 

attack_methods = ['fgsm', 'bim', 'pgd', 'df', 'cw']  
detector = "LayerPFS"  
net = "inat" 

for attack_method in attack_methods:
    extract(IMG_SIZE, CSV_PATH, CLEAN_BASE, ADV_BASE_ROOT, TRAIN_CSV_FOR_MAH,  attack_method, detector, net)

Building stacks from CSV...


Loading images and advs:   8%|████▋                                                   | 54/649 [00:02<00:23, 25.04it/s]

KeyboardInterrupt



In [ ]:
IMG_SIZE = 64
CSV_PATH = Path("../02-AdvGenerate/Evaluate/common_success_details.csv")
CLEAN_BASE = Path(f"../01-CleanModel/Dataset/{IMG_SIZE}x{IMG_SIZE}")
ADV_BASE_ROOT = Path(f"../02-AdvGenerate/generated_images-test/{IMG_SIZE}x{IMG_SIZE}")
TRAIN_CSV_FOR_MAH = Path("../01-CleanModel/Dataset/clean_model_train.csv") 

attack_methods = ['fgsm', 'bim', 'pgd', 'df', 'cw']  
detector = "LayerPFS"  
net = "inat" 

for attack_method in attack_methods:
    extract(IMG_SIZE, CSV_PATH, CLEAN_BASE, ADV_BASE_ROOT, TRAIN_CSV_FOR_MAH,  attack_method, detector, net)

In [ ]:
IMG_SIZE = 128
CSV_PATH = Path("../02-AdvGenerate/Evaluate/common_success_details.csv")
CLEAN_BASE = Path(f"../01-CleanModel/Dataset/{IMG_SIZE}x{IMG_SIZE}")
ADV_BASE_ROOT = Path(f"../02-AdvGenerate/generated_images-test/{IMG_SIZE}x{IMG_SIZE}")
TRAIN_CSV_FOR_MAH = Path("../01-CleanModel/Dataset/clean_model_train.csv") 

attack_methods = ['fgsm', 'bim', 'pgd', 'df', 'cw']  
detector = "LayerPFS"  
net = "inat" 

for attack_method in attack_methods:
    extract(IMG_SIZE, CSV_PATH, CLEAN_BASE, ADV_BASE_ROOT, TRAIN_CSV_FOR_MAH,  attack_method, detector, net)

In [ ]:
IMG_SIZE = 256
CSV_PATH = Path("../02-AdvGenerate/Evaluate/common_success_details.csv")
CLEAN_BASE = Path(f"../01-CleanModel/Dataset/{IMG_SIZE}x{IMG_SIZE}")
ADV_BASE_ROOT = Path(f"../02-AdvGenerate/generated_images-test/{IMG_SIZE}x{IMG_SIZE}")
TRAIN_CSV_FOR_MAH = Path("../01-CleanModel/Dataset/clean_model_train.csv") 

attack_methods = ['fgsm', 'bim', 'pgd', 'df', 'cw']  
detector = "LayerPFS"  
net = "inat" 

for attack_method in attack_methods:
    extract(IMG_SIZE, CSV_PATH, CLEAN_BASE, ADV_BASE_ROOT, TRAIN_CSV_FOR_MAH,  attack_method, detector, net)

# Using memmap - Limited RAM

In [3]:
def extract_characteristics_from_stacks(images, images_advs, model, detector_name, net_name):
    number_images = images.shape[0]

    model_features = model.features
    act_layers= [2,5,9,12,16,19,22,26,29,32,36,39,42]
    fourier_act_layers = [9,16,22,29,36,42]

    print('Extracting ' + detector_name + ' characteristic...')

    use_mmap_out = IMG_SIZE in (512, 1024)
    MEMMAP_DIR = Path("./memmap_cache")
    
    MEMMAP_DIR.mkdir(parents=True, exist_ok=True)

    if detector_name == 'LayerMFS' or detector_name == 'LayerPFS':
        typ = 'MFS' if detector_name == 'LayerMFS' else 'PFS'
        if net_name == 'cif100' and (attack_method in ('cw','df')):
            layers = [42]
        else:
            layers = fourier_act_layers

        sample_t = images[0]
        sample_tensor = sample_t.unsqueeze(0) if isinstance(sample_t, torch.Tensor) and sample_t.dim()==3 else torch.from_numpy(np.asarray(sample_t)).unsqueeze(0)
        sample_norm = cifar_normalize(sample_tensor, net_name)
        with torch.no_grad():
            sample_maps = get_layer_feature_maps(sample_norm.to(device), layers, model_features)
        sample_parts = []
        for m in sample_maps:
            m_single = m[0] if isinstance(m, torch.Tensor) and m.dim()==4 else (m if isinstance(m, torch.Tensor) else torch.from_numpy(np.asarray(m)))
            sample_parts.append(calculate_fourier_spectrum(m_single.cpu(), typ=typ).flatten())
        feat_dim = sum(p.size for p in sample_parts)

        if use_mmap_out:
            mm_path = MEMMAP_DIR / f"char_{detector_name}_{IMG_SIZE}.npy"
            mm_path_adv = MEMMAP_DIR / f"char_{detector_name}_{IMG_SIZE}_adv.npy"
            out_mm = open_memmap(str(mm_path), mode='w+', dtype=np.float32, shape=(number_images, feat_dim))
            out_mm_adv = open_memmap(str(mm_path_adv), mode='w+', dtype=np.float32, shape=(number_images, feat_dim))

            for i in tqdm(range(number_images)):
                s = images[i]
                a = images_advs[i]
                s_t = s.unsqueeze(0) if isinstance(s, torch.Tensor) and s.dim()==3 else torch.from_numpy(np.asarray(s)).unsqueeze(0)
                a_t = a.unsqueeze(0) if isinstance(a, torch.Tensor) and a.dim()==3 else torch.from_numpy(np.asarray(a)).unsqueeze(0)
                s_t = cifar_normalize(s_t, net_name)
                a_t = cifar_normalize(a_t, net_name)
                with torch.no_grad():
                    s_maps = get_layer_feature_maps(s_t.to(device), layers, model_features)
                    a_maps = get_layer_feature_maps(a_t.to(device), layers, model_features)
                s_parts = [calculate_fourier_spectrum(m[0].cpu(), typ=typ).flatten() for m in s_maps]
                a_parts = [calculate_fourier_spectrum(m[0].cpu(), typ=typ).flatten() for m in a_maps]
                out_mm[i, :] = np.hstack(s_parts).astype(np.float32)
                out_mm_adv[i, :] = np.hstack(a_parts).astype(np.float32)

                del s_maps, a_maps, s_parts, a_parts
                if (i+1) % 50 == 0:
                    out_mm.flush()
                    out_mm_adv.flush()
                    gc.collect()

            out_mm.flush(); out_mm_adv.flush()
            return out_mm, out_mm_adv
    else:
        raise ValueError("unknown detector")


In [ ]:
IMG_SIZE = 512
CSV_PATH = Path("../02-AdvGenerate/Evaluate/common_success_details.csv")
CLEAN_BASE = Path(f"../01-CleanModel/Dataset/{IMG_SIZE}x{IMG_SIZE}")
ADV_BASE_ROOT = Path(f"../02-AdvGenerate/generated_images-test/{IMG_SIZE}x{IMG_SIZE}")
TRAIN_CSV_FOR_MAH = Path("../01-CleanModel/Dataset/clean_model_train.csv") 

attack_methods = ['fgsm', 'bim', 'pgd', 'df', 'cw']  
detector = "LayerPFS"  
net = "inat" 

for attack_method in attack_methods:
    extract(IMG_SIZE, CSV_PATH, CLEAN_BASE, ADV_BASE_ROOT, TRAIN_CSV_FOR_MAH,  attack_method, detector, net)

In [ ]:
IMG_SIZE = 1024
CSV_PATH = Path("../02-AdvGenerate/Evaluate/common_success_details.csv")
CLEAN_BASE = Path(f"../01-CleanModel/Dataset/{IMG_SIZE}x{IMG_SIZE}")
ADV_BASE_ROOT = Path(f"../02-AdvGenerate/generated_images-test/{IMG_SIZE}x{IMG_SIZE}")
TRAIN_CSV_FOR_MAH = Path("../01-CleanModel/Dataset/clean_model_train.csv") 

attack_methods = ['fgsm', 'bim', 'pgd', 'df']  
detector = "LayerPFS"  
net = "inat" 

for attack_method in attack_methods:
    extract(IMG_SIZE, CSV_PATH, CLEAN_BASE, ADV_BASE_ROOT, TRAIN_CSV_FOR_MAH,  attack_method, detector, net)

In [4]:
IMG_SIZE = 1024
CSV_PATH = Path("../02-AdvGenerate/Evaluate/common_success_details.csv")
CLEAN_BASE = Path(f"../01-CleanModel/Dataset/{IMG_SIZE}x{IMG_SIZE}")
ADV_BASE_ROOT = Path(f"../02-AdvGenerate/generated_images-test/{IMG_SIZE}x{IMG_SIZE}")
TRAIN_CSV_FOR_MAH = Path("../01-CleanModel/Dataset/clean_model_train.csv") 

attack_methods = ['cw']  
detector = "LayerPFS"  
net = "inat" 

for attack_method in attack_methods:
    extract(IMG_SIZE, CSV_PATH, CLEAN_BASE, ADV_BASE_ROOT, TRAIN_CSV_FOR_MAH,  attack_method, detector, net)

Building stacks from CSV...


Loading images and advs: 100%|███████████████████████████████████████████████████████| 649/649 [02:52<00:00,  3.77it/s]


Built stacks: 649 pairs (skipped 0).
Stack shapes: torch.Size([649, 3, 1024, 1024])
Loading model...
Loaded checkpoint ../01-CleanModel/Models/1024x1024/best_min_acc_vgg16_1024x1024_Model-2.pth (strict=True).
Extracting LayerPFS characteristic...


100%|██████████████████████████████████████████████████████████████████████████████| 649/649 [5:30:18<00:00, 30.54s/it]


Saved characteristics: data\characteristics\1024\inat_cw_LayerPFS
Saved characteristics adv: data\characteristics\1024\inat_cw_LayerPFS_adv
Done.
